In [1]:
import requests
import json
import pandas as pd

# Overpass API URL
overpass_url = "http://overpass-api.de/api/interpreter"

# Overpass Query to get gas stations in Ontario
overpass_query = """
[out:json][timeout:60];
area[name="Ontario"]["ISO3166-2"="CA-ON"]->.searchArea;
(
  node["amenity"="fuel"](area.searchArea);
  way["amenity"="fuel"](area.searchArea);
  relation["amenity"="fuel"](area.searchArea);
);
out center;
"""

# Fetch data from Overpass API
response = requests.get(overpass_url, params={'data': overpass_query})

if response.status_code == 200:
    data = response.json()

    # Process the data
    gas_stations = []
    for element in data["elements"]:
        lat = element.get("lat", element.get("center", {}).get("lat"))
        lon = element.get("lon", element.get("center", {}).get("lon"))
        name = element.get("tags", {}).get("name", "Unknown")
        brand = element.get("tags", {}).get("brand", "Unknown")

        gas_stations.append({
            "Name": name,
            "Brand": brand,
            "Latitude": lat,
            "Longitude": lon
        })

    # Convert to DataFrame
    df = pd.DataFrame(gas_stations)

    # Save to CSV
    df.to_csv("ontario_gas_stations.csv", index=False)
    print("Data saved as 'ontario_gas_stations.csv'")

    # Display first 5 rows
    print(df.head())

else:
    print(f"Error: Unable to fetch data (HTTP {response.status_code})")


Data saved as 'ontario_gas_stations.csv'
           Name         Brand   Latitude  Longitude
0          Esso          Esso  43.408347 -80.328698
1         Shell         Shell  43.411170 -80.308820
2          Esso          Esso  43.459499 -80.134146
3  Petro-Canada  Petro-Canada  43.493588 -79.991969
4         Shell         Shell  44.266705 -76.498106
